## Chargement de données vélib

In [ ]:
from velibconnector import VelibConnector
import pandas as pd
import plotly.express as px
import numpy as np
import math
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

cmd = """
SELECT station, lat, lon,
    delta, bikes, capacity, name,
        dt from velib_all
        where dt >= poll_dt::date and dt>'2024-12-05'
"""
df = VelibConnector(cmd).to_pandas()
# Gardons la copie d'origine au cas où
df_saved = df.copy()
print(f'{len(df)} rows chargés pour la période de {df.dt.min().date()} à {df.dt.max().date()}')


PostgreSQL connection closed.
2139296 rows chargés pour la période de 2024-12-05 à 2025-02-07


## Prétraitement et transformation

In [ ]:
df = df_saved.copy()

In [ ]:
# On a les données horodataires
df['datehour'] = df.dt.dt.floor('h')

In [ ]:
print('Drop NaN values:', df.isna().sum().sum())
df.dropna()
full_duplicates = df.duplicated(['dt', 'bikes', 'capacity', 'station']).sum()
print(f"Doublons des valeurs realtime à supprimer: {full_duplicates}, {round(full_duplicates/len(df)*100, 2)}%")
df.drop_duplicates(['dt', 'bikes', 'capacity', 'station'], inplace=True)
calc_duplicated = df.duplicated(['datehour', 'station']).sum()
print(f"Enregistrements répétées par heure/station: {calc_duplicated}, {round(calc_duplicated/len(df)*100, 2)}%")
# Analyse et nettoyage doublons heure/station
display(df[df.duplicated(['datehour', 'station'], keep=False)].sort_values('dt').head(10))


Drop NaN values: 0
Doublons des valeurs realtime à supprimer: 3551, 0.17%
Enregistrements répétées par heure/station: 23804, 1.11%


,station,lat,lon,delta,bikes,capacity,name,dt,datehour
1844510,213706622,48.86053161154489,2.4091592812626543,0,3,23,Vitruve - Davout,2024-12-05 02:20:27,2024-12-05 02:00:00
1845949,213706622,48.86053161154489,2.4091592812626543,0,3,23,Vitruve - Davout,2024-12-05 02:46:53,2024-12-05 02:00:00
1852780,2515976231,48.901365,2.212693,-1,1,20,Nanterre - Université,2024-12-05 08:20:14,2024-12-05 08:00:00
1854220,2515976231,48.901365,2.212693,0,1,20,Nanterre - Université,2024-12-05 08:57:00,2024-12-05 08:00:00
1863336,36391,48.869789,2.326347,0,17,22,Godot de Mauroy - Madeleine,2024-12-05 15:13:56,2024-12-05 15:00:00
1864776,36391,48.869789,2.326347,-4,13,22,Godot de Mauroy - Madeleine,2024-12-05 15:13:56,2024-12-05 15:00:00
1866216,36391,48.869789,2.326347,-7,6,22,Godot de Mauroy - Madeleine,2024-12-05 15:13:56,2024-12-05 15:00:00
1867656,36391,48.869789,2.326347,-3,3,22,Godot de Mauroy - Madeleine,2024-12-05 15:13:56,2024-12-05 15:00:00
1865986,35106544,48.82479134498032,2.336124926805496,-2,3,55,René Coty - Parc Montsouris,2024-12-05 17:22:28,2024-12-05 17:00:00
1867426,35106544,48.82479134498032,2.336124926805496,-1,2,55,René Coty - Parc Montsouris,2024-12-05 17:24:44,2024-12-05 17:00:00


In [ ]:
# Il s'agit des données consecutives. On peut les résumer
def first(x):
    return x.iloc[0]
def last(x):
    return x.iloc[-1]
duplicates = df[df.duplicated(['datehour', 'station'], keep=False)]
duplicates = duplicates.sort_values('dt').groupby(['datehour', 'station']).agg({
    'lat' : first,
    'lon' : first,
    'delta' : 'sum',
    'bikes' : last,
    'capacity' : last,
})
df = df.set_index(['datehour', 'station'])
df.update(duplicates)
df = df.reset_index()
df.drop_duplicates(['datehour', 'station'], inplace=True)
calc_duplicated = df.duplicated(['datehour', 'station']).sum()
print(f"Enregistrements répétées par heure/station après merge: {calc_duplicated}, {round(calc_duplicated/len(df)*100, 2)}%")


Enregistrements répétées par heure/station après merge: 0, 0.0%


In [ ]:
px.box(df.station.value_counts(), 'count', labels={'count' : 'Stations'}, title='Distribution de nombre de stations et présence des outliers').show()
datehour_df = df.groupby('datehour').station.count().reset_index()
px.line(x=datehour_df.datehour, y=datehour_df['station'], labels={'datehour' : 'Date-heure', 'station' : 'Nb de stations par heure'}, title='Nombre de stations connues par heure').show()
px.box(datehour_df, 'station', labels={'station' : 'Nb de station par heure'}, title='Distribution de nombre de stations par heure').show()

In [ ]:
from velibdslib import detect_outliers

clean_df = df.copy()

print('Stations outliers:')
blacklist_stations = [] + detect_outliers(clean_df.station.value_counts(), k=3)
print('Hours outliers:')
blacklist_hours = [] + detect_outliers(clean_df.datehour.value_counts(), k=3)
# blacklist_hours = detect_outliers(clean_df.groupby('datehour').station.count(), k=1350, method='hard')

total_rows = len(clean_df)
drop_mask = clean_df.station.isin(blacklist_stations) | clean_df.datehour.isin(blacklist_hours)
dropped_rows = drop_mask.sum()
print(f'Il faut supprimer {dropped_rows} lignes soit {round(dropped_rows/total_rows*100, 2)}% sur {total_rows} ligne au total.')
# clean_df = clean_df[~drop_mask]
print("Comme c'est une perte assez élevé faison la clusterisation pour niveler le probleme de \"bruit\" de petites stations.")


Stations outliers:
Valeur min-max: 19 - 1478
Seuils de outliers: 1430.0 - 1493.0
Nombre de valeurs total: 1460
Grands outliers: 0 ou 0.0%
Petits outliers: 93 ou 6.37%
Hours outliers:
Valeur min-max: 273 - 1453
Seuils de outliers: 1416.0 - 1468.5
Nombre de valeurs total: 1483
Grands outliers: 0 ou 0.0%
Petits outliers: 208 ou 14.03%
Il faut supprimer 371524 lignes soit 17.59% sur 2111941 ligne au total.
Comme c'est une perte assez élevé faison la clusterisation pour niveler le probleme de "bruit" de petites stations.


In [ ]:
from velibdslib import get_station_clusters

stations = df[['station', 'lat', 'lon', 'name']].drop_duplicates()
# n_clusters : 73 le meilleur, meilleurs des plus grands = 119 et 142
stations['labels'] = get_station_clusters(stations, KMeans(n_clusters=119, random_state=0))
fig = px.scatter_map(stations, 'lat', 'lon', color='labels', size_max=20, color_continuous_scale='Viridis', zoom=10, width=800, height=600).show()


In [ ]:
from velibdslib import draw_kmeans_silhouette

draw_kmeans_silhouette(stations)

In [ ]:
# La taille réelle des clusters
from velibdslib import max_distance_in_cluster
max_distances = []
for l in sorted(stations['labels'].unique()):
    max_distances.append(int(max_distance_in_cluster(stations[stations.labels == l])))
max_distances = [d for d in max_distances if d != -1]
print(f'Taille mediane des clusters: {np.median(max_distances)} m., max: {np.max(max_distances)} m., min: {np.min(max_distances)} m.')
 

Taille mediane des clusters: 1645.0 m., max: 3010 m., min: 516 m.


In [ ]:
## Creation de zone de clusters pour Viz
from velibdslib import get_border, points_to_geo_json, draw_stations_choroplethmap_scatter

borders = []
for l in sorted(stations.labels.unique()):
    borders.append(get_border(stations[stations.labels==l], l))
geo = points_to_geo_json(borders)
draw_stations_choroplethmap_scatter(geo, stations)

In [ ]:
# On met les clusters dans le dataset
clean_df = pd.merge(clean_df, stations[['station', 'labels']], on='station', how='left')

In [ ]:
px.box(clean_df.labels.value_counts(), 'count', labels={'count' : 'Clusters'}, title='Distribution de nombre de clusters et présence des outliers').show()
datehour_df = clean_df.groupby('datehour').labels.nunique().reset_index()
px.line(x=datehour_df.datehour, y=datehour_df['labels'], labels={'x' : 'Date-heure', 'y' : 'Nb de clusters par heure'}, title='Nombre de labels connues par heure').show()
px.box(datehour_df, 'labels', labels={'labels' : 'Nb de clusters par heure'}, title='Distribution de nombre de clusters par heure').show()

In [ ]:
# Nettoyage des heures outliers
datehour_df = clean_df.groupby('datehour').labels.nunique()
q = datehour_df.quantile([0.25, 0.75]).to_list()
# outliers comme q1 - delta(q3,q1)*1.5 et q3 + delta(q3,q1)*1.5
q_margin = (q[1] - q[0]) * 1.5
seuil_bas = q[0] - q_margin
seuil_haut = q[1] + q_margin
print('Seuils de outliers:', seuil_bas, '-', seuil_haut)
print("Nombre de clusters:", clean_df.labels.nunique())
print('Max stations par heure:', datehour_df.max())
print('Min stations par heure:', datehour_df.min())
top_outliers = datehour_df[datehour_df>seuil_haut]
top_outliers_count = top_outliers.count()
bottom_outliers = datehour_df[datehour_df<seuil_bas]
bottom_outliers_count = bottom_outliers.count()
total_hours = clean_df.datehour.nunique()
print("Nombre d'heures:", total_hours)
print('Nombre de top outliers:', top_outliers_count, f"{round(top_outliers_count/total_hours*100, 2)}%")
print('Nombre de bottom outliers:', bottom_outliers_count, f"{round(bottom_outliers_count/total_hours*100, 2)}%")
blacklist = bottom_outliers.index.to_list() + top_outliers.index.to_list()
total_rows = len(clean_df)
dropped_rows = len(clean_df[clean_df.datehour.isin(blacklist)])
print(f'A enlever {len(blacklist)} outliers, {dropped_rows} lignes soit {round(dropped_rows/total_rows*100, 2)}% de {total_rows} lignes en totale.')
manual_cut = seuil_bas - 1
blacklist = datehour_df[datehour_df<manual_cut].index.to_list()
dropped_rows = len(clean_df[clean_df.datehour.isin(blacklist)])
print(f'Si on coupe manuellement les heures avec moins de {manual_cut} clusters on supprime {len(blacklist)} outliers et {dropped_rows} lignes soit {round(dropped_rows/total_rows*100, 2)}% de {total_rows} ligne en totale.')

clean_df = clean_df[~clean_df.datehour.isin(blacklist)]

Seuils de outliers: 119.0 - 119.0
Nombre de clusters: 119
Max stations par heure: 119
Min stations par heure: 89
Nombre d'heures: 1483
Nombre de top outliers: 0 0.0%
Nombre de bottom outliers: 248 16.72%
A enlever 248 outliers, 346461 lignes soit 16.4% de 2111941 lignes en totale.
Si on coupe manuellement les heures avec moins de 118.0 clusters on supprime 10 outliers et 7755 lignes soit 0.37% de 2111941 ligne en totale.


In [ ]:
# Statistique après le traitement
datehour_df = clean_df.groupby('datehour').labels.nunique().reset_index()
px.line(x=datehour_df.datehour, y=datehour_df['labels'], labels={'x' : 'Date-heure', 'y' : 'Nb de clusters par heure'}, title='Nombre de clusters connues par heure').show()
px.box(datehour_df, 'labels', labels={'labels' : 'Nb de clusters par heure'}, title='Distribution de nombre de clusters par heure').show()
datehour_df_stations = clean_df.groupby('datehour').station.nunique().reset_index()
px.box(datehour_df_stations, 'station', labels={'station' : 'Nb de station par heure'}, title='Distribution de nombre de stations par heure').show()
px.line(datehour_df_stations, 'datehour', 'station', labels={'datehour' : 'Date-heure', 'station' : '# de stations'}, title='Nombre de stations connues par heure').show()



In [ ]:
print(f"On a obtenu {len(clean_df):,} ligne soit {round(len(clean_df)/len(df_saved), 2)}% du dataset original.")

On a obtenu 2,104,186 ligne soit 0.98% du dataset original.


# DataViz

In [ ]:

src_df = clean_df.groupby('datehour')[['delta', 'bikes', 'capacity']].sum().reset_index()
datehour_df = pd.concat([src_df[['datehour', 'delta']], src_df[['datehour', 'bikes']], src_df[['datehour', 'capacity']]])
datehour_df['value'] = datehour_df.bfill(axis=1)['delta']
datehour_df['line'] = datehour_df.apply(lambda row: 'Delta' if not math.isnan(row.delta) else 'Vélos' if not math.isnan(row.bikes) else 'Capacity', axis=1 )
px.line(datehour_df, x='datehour', y='value', color='line', 
        height=800, title="Dynamique d'utilisation des stations",
        labels={'line': 'Data', 'datehour' : 'Date-Heure', 'value' : 'Sommes'}).show()



In [ ]:
from velibdslib import draw_fig
clean_df['year'] = clean_df.dt.dt.year
clean_df['month'] = clean_df.dt.dt.month
clean_df['week'] = clean_df.dt.dt.isocalendar().week
clean_df['weekday'] = clean_df.dt.dt.weekday + 1
clean_df['date'] = clean_df.dt.dt.date

usage_weekly = clean_df.groupby(['weekday', 'datehour']).delta.sum().reset_index().groupby(['weekday']).aggregate(
        delta_min=pd.NamedAgg(column="delta", aggfunc="min"),
        delta_max=pd.NamedAgg(column="delta", aggfunc="max"),
        delta_mean=pd.NamedAgg(column="delta", aggfunc="mean"),
).reset_index()

draw_fig(usage_weekly, [
    {'x' : 'weekday', 'y' : 'delta_min', 'fill' : 'rgba(100, 0, 80, 0.2)', 'color' : 'red', 'name' : 'Min'},
    {'x' : 'weekday', 'y' : 'delta_mean', 'fill' : None, 'color' : 'black', 'name' : 'Moyenne'},
    {'x' : 'weekday', 'y' : 'delta_max', 'fill' : 'rgba(100, 0, 80, 0.2)', 'color' : 'red', 'name' : 'Max'},
],
title = 'Utilisation horodatée des vélos par jour de semaine',
legend = 'Somme de vélos partis/rendus',
xaxis={'title' : 'Jour de semaine', 'tickvals' : list(range(1,8)), 'ticktext' : ['Lun', 'Mar', 'Mer', 'Jeu', 'Ven', 'Sam', 'Dim']}
)


In [ ]:
clean_df['hour'] = clean_df.dt.dt.hour

usage_hourly = clean_df.groupby(['datehour', 'hour']).delta.sum().reset_index().groupby(['hour']).aggregate(
        delta_min=pd.NamedAgg(column="delta", aggfunc="min"),
        delta_max=pd.NamedAgg(column="delta", aggfunc="max"),
        delta_mean=pd.NamedAgg(column="delta", aggfunc="mean"),
).reset_index()

draw_fig(usage_hourly, [
    {'x' : 'hour', 'y' : 'delta_min', 'fill' : None, 'color' : 'blue', 'name' : 'Min'},
    {'x' : 'hour', 'y' : 'delta_mean', 'fill' : 'rgba(0, 80, 100, 0.2)', 'color' : 'black', 'name' : 'Moyenne'},
    {'x' : 'hour', 'y' : 'delta_max', 'fill' : 'rgba(100, 0, 80, 0.2)', 'color' : 'red', 'name' : 'Max'},
],
title = 'Utilisation de vélos par heure',
legend = 'Somme de vélos partis/rendus',
xaxis={'title' : 'Heure', 'dtick': 1, 'range' : [0, 23]}
)



In [ ]:
clean_df['taken'] = clean_df.delta.apply(lambda x: x if x < 0 else 0)
clean_df['received'] = clean_df.delta.apply(lambda x: x if x > 0 else 0)
usage_hourly_sep = clean_df.groupby(['datehour', 'hour']).agg({'taken' : 'sum', 'received' : 'sum'}) \
    .reset_index().groupby(['hour']).agg({'taken' : 'mean', 'received' : 'mean'}).reset_index()


draw_fig(usage_hourly_sep, [
    {'x' : 'hour', 'y' : 'received', 'name' : 'Vélos arrivés', 'color' : 'green', 'fill' : 'rgba(0, 100, 0, 0.2)'},
    {'x' : 'hour', 'y' : 'taken', 'name' : 'Vélos sortis', 'color' : 'red', 'fill' : 'rgba(100, 0, 0, 0.2)'}
    ], title='Flux moyen des vélos par heure sur tout le réseaux', xaxis={'title' : 'Heure', 'dtick' : 1})


In [ ]:
clean_df['empty'] = (clean_df.bikes == 0)

empty_hourly = clean_df.groupby(['datehour', 'hour']).empty.sum().reset_index().groupby(['hour']).aggregate(
        empty_min=pd.NamedAgg(column="empty", aggfunc="min"),
        empty_max=pd.NamedAgg(column="empty", aggfunc="max"),
        empty_mean=pd.NamedAgg(column="empty", aggfunc="mean"),
).reset_index()

draw_fig(empty_hourly, [
    {'x' : 'hour', 'y' : 'empty_min', 'fill' : None, 'color' : 'red', 'name' : 'Min'},
    {'x' : 'hour', 'y' : 'empty_mean', 'fill' : 'rgba(100, 0, 80, 0.2)', 'color' : 'black', 'name' : 'Moyenne'},
    {'x' : 'hour', 'y' : 'empty_max', 'fill' : 'rgba(100, 0, 80, 0.2)', 'color' : 'red', 'name' : 'Max'},
],
title = 'Nombre de stations vides par heure',
legend = 'Stations vides',
xaxis={'title' : 'Heure', 'dtick': 1, 'range' : [0, 23]},
yaxis = {'dtick' : 20} 
)
